In [3]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf

from core.data import make_batches, positional_encoding, tokenizers
from core.transformer import Transformer, MiniTransformer
from core.scheduler import CustomSchedule
from core.callbacks import get_callbacks

In [4]:
num_layers = 6
d_model = 512
dff = 2048
num_heads = 8
dropout_rate = 0.1
exp_path = "./experiments/train"
EPOCHS = 20
BATCHSIZE = 64

In [5]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
test_examples = examples['test']
test_batches = make_batches(test_examples, batchsize=BATCHSIZE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/51785 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1193 [00:00<?, ? examples/s]

Shuffling ted_hrlr_translate-validation.tfrecord...:   0%|          | 0/1193 [00:00<?, ? examples/s]

Generating test examples...:   0%|          | 0/1803 [00:00<?, ? examples/s]

Shuffling ted_hrlr_translate-test.tfrecord...:   0%|          | 0/1803 [00:00<?, ? examples/s]

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [7]:
# Optimizer
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

# Loss Function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                            reduction='none')

transformer = MiniTransformer(num_layers=num_layers,
                              d_model=d_model,
                              num_heads=num_heads,
                              dff=dff,
                              input_vocab_size=tokenizers.en.get_vocab_size(),
                              target_vocab_size=tokenizers.en.get_vocab_size(),
                              pe_input=1000,
                              rate=0.1)
transformer.compile(optimizer=optimizer, 
                    loss_function=loss_object,
                    metrics=['accuracy', loss_object])

transformer.model(BATCHSIZE)

transformer.evaluate(test_batches.take(1))

1/1 [==============================] - 15s 15s/step - accuracy: 0.0000e+00 - sparse_categorical_crossentropy: 9.1458


[0.0, 9.14582347869873]

In [8]:
expdir = '{}/train_model.h5'.format(exp_path)
transformer.load_weights(expdir)

In [9]:
prediction = transformer.predict(test_batches.take(1))

In [10]:
y_pred = prediction[0]
y_true = prediction[1]

In [11]:
for t, k in zip(y_pred, y_true):
    print(k.decode('utf-8'))
    print('---')
    print(t.decode('utf-8'))
    print(':'*10)

in the late ' 70s , there were a group of swedish orthopedic surgeons who met at their annual meeting , and they were discussing the different procedures they used to operate hip surgery .
---
and fact u 2000 80s , i were no lot of peopleco individuals people twomarss , were in the core openual rate with and they were helded the same times of were to be .p .
::::::::::
( applause ) it ' s a disaster ! that ' s not a language .
---
and applause ) ' s a little . ' s a a big of
::::::::::
the next day , all the artists were invited to join mandela in robben island , where he was going to give a conference to the world ' s press , standing in front of his former prison cell .
---
and first one , i of kids , dead to the the , theadbroy . , and they was going to get him speech in the hospital of s u . and in the of him house friends . phones
::::::::::
one goes extinct for the other to survive .
---
and of back . the next reason the .
::::::::::
they ' re as smart as chimpanzees in some resp